# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# pip install pandas holoviews hvplot

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Output_Data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,-8.91,98,39,5.88,RU,2023-11-25
1,1,port elizabeth,-33.9180,25.5701,68.29,85,100,5.75,ZA,2023-11-25
2,2,albany,42.6001,-73.9662,39.56,52,28,3.00,US,2023-11-25
3,3,kingston,17.9970,-76.7936,84.20,60,80,5.10,JM,2023-11-25
4,4,constantia,44.1833,28.6500,39.88,91,100,28.95,RO,2023-11-25


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
# %%capture --no-display

# Configure the map plot
city_data_map_plot = city_data_df.hvplot.points(
    x ='Lng',
    y= 'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.63,
    color = 'City')

# Display the map
city_data_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_df = city_data_df[(city_data_df['Max Temp']<=80) & (city_data_df['Max Temp']>15)
                        & (city_data_df['Humidity']<=60) & (city_data_df['Humidity']>20)]

# Drop any rows with null values
ideal_df.dropna()
ideal_df.head(15)

# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,albany,42.6001,-73.9662,39.56,52,28,3.00,US,2023-11-25
20,20,misratah,32.3754,15.0925,68.31,58,59,29.39,LY,2023-11-25
21,21,teguise,29.0605,-13.5640,66.42,56,20,13.80,ES,2023-11-25
23,23,ushuaia,-54.8000,-68.3000,58.66,31,75,20.71,AR,2023-11-25
24,24,cidreira,-30.1811,-50.2056,68.79,59,5,10.07,BR,2023-11-25
28,28,puerto natales,-51.7236,-72.4875,59.45,44,40,40.29,CL,2023-11-25
31,31,bethel,41.3712,-73.4140,40.01,50,0,4.61,US,2023-11-25
32,32,saint clairsville,40.0806,-80.9001,42.31,54,23,4.00,US,2023-11-25
34,34,mumbwa,-14.9783,27.0619,79.23,42,91,9.64,ZM,2023-11-25
35,35,newman,37.3138,-121.0208,52.23,39,0,5.68,US,2023-11-25


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# can also use go back to hotel_df.loc[:, 'Hotel Name] = ''

hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

C:\Users\peter\AppData\Local\Temp\ipykernel_14180\1856896625.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,52,
20,misratah,LY,32.3754,15.0925,58,
21,teguise,ES,29.0605,-13.5640,56,
23,ushuaia,AR,-54.8000,-68.3000,31,
24,cidreira,BR,-30.1811,-50.2056,59,
...,...,...,...,...,...,...
550,mokpo,KR,34.7936,126.3886,47,
554,butte,US,39.6499,-121.5677,42,
555,la palma,US,33.8464,-118.0467,50,
563,rexburg,US,43.8260,-111.7897,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "categories": "accommodation.hotel"
}

# accommodation.hotel

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude =  row['Lat']
    longitude =  row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude 
    # to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


Starting hotel search
albany - nearest hotel: No hotel found
misratah - nearest hotel: الفندق السياحى
teguise - nearest hotel: Finca Las Laderas
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
cidreira - nearest hotel: Hotel Castelo
puerto natales - nearest hotel: Dorotea Patagonia Hostel
bethel - nearest hotel: Hampton Inn Danbury
saint clairsville - nearest hotel: Residence Inn by Marriott Wheeling-St. Clairsville, OH
mumbwa - nearest hotel: La Hacienda
newman - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
nouadhibou - nearest hotel: Hotel valencia
aral - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
muynoq - nearest hotel: Lala
tejen - nearest hotel: Tejen Myhmanhanasy
grand falls-windsor - nearest hotel: Hotel Robin Hood
broni - nearest hotel: Hotel Italia
klyuchi - nearest hotel: No hotel found
bakanas - nearest hotel: Карой
hamilton - nearest hotel: North Vis

In [55]:
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,52,No hotel found
20,misratah,LY,32.3754,15.0925,58,الفندق السياحى
21,teguise,ES,29.0605,-13.5640,56,Finca Las Laderas
23,ushuaia,AR,-54.8000,-68.3000,31,Apart Hotel Aires del Beagle
24,cidreira,BR,-30.1811,-50.2056,59,Hotel Castelo
...,...,...,...,...,...,...
550,mokpo,KR,34.7936,126.3886,47,백제관광호텔
554,butte,US,39.6499,-121.5677,42,No hotel found
555,la palma,US,33.8464,-118.0467,50,La Quinta Inn & Suites by Wyndham Buena Park
563,rexburg,US,43.8260,-111.7897,48,Quality Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
# %%capture --no-display
# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    x ='Lng',
    y= 'Lat',
    geo = True,
    tiles = 'OSM',
#     frame_width = 700,
#     frame_height = 500,
#     size = 'Humidity',
    scale = 0.63,
    color = 'Hotel Name')

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)